In [3]:
import numpy as np
import pandas as pd
import random



In [24]:
def map_words(row):
    word_to_int = {'beginner': 1, 'intermediate': 2, 'advanced': 3, 'pro': 4, 'challenging': 5}
    words = row.split()
    ints = [word_to_int[word] for word in words]
    return max(ints)

def loadPandasDataFromCsv(path):
    df = pd.read_csv(path)
    df = df.applymap(lambda x: x.lower() if type(x) == str else x)
    df = df.applymap(lambda x: x.strip() if type(x) == str else x)
    df['Muscles Worked'] = df['Muscles Worked'].str.replace('and|\\sand\\s',' ',regex=True)

    # Use Series.replace with regex=True
    df['Muscles Worked'] = df['Muscles Worked'].replace({r'quads(\s|$)': r'quadriceps\1',r'full-body(\s|$)': r'full_body\1',r'hamstrings(\s|$)': r'hamstring\1',
                    r'shoulders(\s|$)': r'shoulder\1',r'glutes(\s|$)': r'glute\1',r'hams(\s|$)': r'hamstring\1'}, regex=True)
    df['Difficulty'] = df['Difficulty'].replace({'beginner to intermediate': 'beginner intermediate',' beginner to intermediate': 'beginner intermediate',
                                                'intermediate to advanced': 'intermediate advanced','beginner to advanced':'beginner intermediate advanced',
                                                ' beginner to advanced':'beginner intermediate advanced' })


    df['Difficulty'] = df['Difficulty'].apply(map_words)

    return df

def processWorkoutData(equipment, skills, duration):
    # Your processing logic here
    result = {'equipment': equipment, 'skills':skills, "duration": duration}
    print(result)
    return result

def chooseExercice(dataframe, muscles_list, equipment_list, difficulty):
    list_exo = []
    random.shuffle(muscles_list)
    for muscle in muscles_list:

        exercice = list(dataframe.loc[(dataframe['Equipment'].isin(equipment_list)) 
                                                       & (dataframe['Difficulty'] <= difficulty) & (dataframe['Muscles Worked'] == muscle)]['Name'])
        #print(exercice)
        if not exercice == []:
            exercice_by_muscle = random.choice(exercice)
            list_exo.append(exercice_by_muscle)
        #else: print('no exercice for ',muscle, 'with this equipment and difficulty')
    random.shuffle(list_exo)
    return list_exo

def weightExerciceGestion(dataframe, exercices_list, user):
    list_exercice_with_weight = []
    equiment_list = []
    for exercice in exercices_list:
        equipment = dataframe.loc[(dataframe['Name'] == exercice)]['Equipment']
        equiment_list.append(equipment)
        print(equipment)
        #if user.performance[exercice] != "" :
        #    list_exercice_with_weight.append((exercice, user.performance[exercice]))
        #elif user.level == 'beginner':
            
        #elif user.level == 'intermediate':
        #elif user.level == 'beginner':
    return list_exercice_with_weight

def wodComposition(dataframe, num_exercices_per_composant, choosing_exercice):
    type_wod=['emom', 'amrap', 'fortime', 'tabata']
    workout_compo = {}
    duration = 0
    exercices_list = random.sample(choosing_exercice, num_exercices_per_composant + int(random.uniform(-1,1)))
    #list_exercice_with_weight = weightExerciceGestion(exercices_list, user)

    wod_type = random.choice(type_wod)
    if wod_type =='emom':
        duration = random.choice(np.arange(6, 16, num_exercices_per_composant))
    elif wod_type=='amrap':
        duration = random.choice(np.arange(3, 14, num_exercices_per_composant))
    elif wod_type=='fortime':
        duration = random.uniform(0,100)
        #print('no duration limit')
    elif wod_type=='tabata':
        duration = random.choice(np.arange(6, 16, num_exercices_per_composant))
        pause = random.choice(np.arange(10, 25, 5))
        duration = (duration, pause)

    workout_compo[wod_type] = [exercices_list, duration]
    return workout_compo

def generateCompleteWod(user, dataframe, number_composant, muscles_list, equipment_list, difficulty):
    # Add user-specific attributes to the workout
    workout = {
        'user_id': user['id'],
        'age': user['age'],
        'weight': user['weight'],
        'height': user['height'],
        'fitness_level': user['fitness_level'],
    }

    # Generate workout components
    for i in range(number_composant):
        num_exercises = random.randint(1, number_composant)
        choosing_exercice = chooseExercice(dataframe, muscles_list, equipment_list, difficulty)
        workout_component = wodComposition(dataframe, number_composant, choosing_exercice)

        # Add workout components and progression to the workout dictionary
        for wod_type, (exercises, duration) in workout_component.items():
            workout[f'wod_type_{i}'] = wod_type
            workout[f'exercises_{i}'] = ', '.join(exercises)
            workout[f'duration_{i}'] = duration

            for j, exercise in enumerate(exercises):
                workout[f'exercise_{i}_{j}_weight'] = user['progression'][exercise]['weight']
                workout[f'exercise_{i}_{j}_reps'] = user['progression'][exercise]['reps']

    return workout


def generate_user_attributes(num_users):
    users = []
    for i in range(num_users):
        age = random.randint(18, 65)
        weight = random.uniform(45, 150)
        height = random.uniform(150, 200)
        fitness_level = random.choice(['beginner', 'intermediate', 'advanced'])

        # Initialize weight and repetition progression for each exercise
        progression = {exercise: {'weight': random.uniform(10, 100), 'reps': random.randint(1, 20)} for exercise in dataframe['Name'].unique()}

        user = {
            'id': i,
            'age': age,
            'weight': weight,
            'height': height,
            'fitness_level': fitness_level,
            'progression': progression,
        }
        users.append(user)
    return users




In [25]:
import pandas as pd

# Generate synthetic user data and workout sessions
num_users = 100
num_workouts_per_user = 10
dataframe = loadPandasDataFromCsv("../data-science/dataset/dataset_exercices_crosstraining.csv")
number_composant = 3
muscles_list = list(set(dataframe['Muscles Worked'].str.split(' ').sum()))[1:]
equipment_list = dataframe['Equipment'].unique()
difficulty = 3

users = generate_user_attributes(num_users)
workouts = []

for user in users:
    for _ in range(num_workouts_per_user):
        workout = generateCompleteWod(user, dataframe, number_composant, muscles_list, equipment_list, difficulty)
        workouts.append(workout)

# Create a DataFrame from the list of workouts
workout_df = pd.DataFrame(workouts)


In [26]:
workout_df.head()

,user_id,age,weight,height,fitness_level,wod_type_0,exercises_0,duration_0,exercise_0_0_weight,exercise_0_0_reps,...,exercise_1_2_reps,wod_type_2,exercises_2,duration_2,exercise_2_0_weight,exercise_2_0_reps,exercise_2_1_weight,exercise_2_1_reps,exercise_2_2_weight,exercise_2_2_reps
0,0,30,90.071923,170.23912,advanced,emom,"dumbbell overhead press, box jumps, pistol squat",12,87.811573,4,...,16,amrap,"chin-ups, dragon fly, plank and its variations",12,67.915018,16,91.058916,18,43.812505,16
1,0,30,90.071923,170.23912,advanced,emom,"barbell rowing, barbell bench press, box step-up",6,72.668893,3,...,9,fortime,"kb goblet squats, box step-up, squat and its v...",99.14051,54.064700,15,37.567071,10,23.245737,8
2,0,30,90.071923,170.23912,advanced,amrap,"plank and its variations, turkish get-up, chin...",12,43.812505,16,...,2,amrap,"bardips, barlsit, dumbbell squat",9,13.723457,19,77.844362,17,93.529186,16
3,0,30,90.071923,170.23912,advanced,fortime,"single-leg romanian deadlift, ropeclimbing, tu...",13.510882,34.642066,10,...,19,fortime,"barbell rowing, pistol squat, box step-up",80.929245,72.668893,3,31.942977,4,37.567071,10
4,0,30,90.071923,170.23912,advanced,fortime,"barbell rowing, tuck ups, kettlebell good morning",96.94867,72.668893,3,...,18,emom,"handstand push-up, box jumps, kb goblet squats",9,80.556931,20,92.006733,1,54.064700,15


In [4]:
num_users = 1000

ages = np.random.randint(16, 61, num_users)
weights = np.random.normal(70, 20, num_users)  # mean 70 kg, standard deviation 10 kg
heights = np.random.normal(1.7, 0.5, num_users)  # mean 1.7 m, standard deviation 0.1 m
fitness_levels = np.random.randint(1, 6, num_users)

users_df = pd.DataFrame({'age': ages, 'weight': weights, 'height': heights, 'fitness_level': fitness_levels})


,age,weight,height,fitness_level
0,45,71.883782,1.638242,4
1,37,72.589991,1.537375,4
2,59,76.658312,1.611752,4
3,58,61.673522,1.638902,3
4,40,70.790930,1.684277,1
